In [17]:
'''
功能: 用TEJ的資料初始化資料庫
讀取檔案路徑: 
1) ../../data/market-1101-1211
2) ../../data/mcounter-1101-1208
'''

import csv
import pprint
import datetime
import copy
import re
from pymongo.mongo_client import MongoClient

pp = pprint.PrettyPrinter(indent = 4)

filename = '../../data/counter-1101-1208.csv'
dateFormat = '%Y/%m/%d'

### hisprice per stock save into MongoDB ###
p_dict = {
    'code': None,
    'history': []
}

### db config ###
client = MongoClient()
db = client['Stock']
stockCollec = db['price']

def appendHis(p_dict, line):
    p_dict['history'].append({
                'date': datetime.datetime.strptime(line['年月日'], dateFormat),
                'open': float(re.sub(',', '', line['開盤價(元)'])),
                'high': float(re.sub(',', '', line['最高價(元)'])),
                'low': float(re.sub(',', '', line['最低價(元)'])),
                'close': float(re.sub(',', '', line['收盤價(元)'])),
                'volumn': float(re.sub(',', '', line['成交量(千股)']))
            })

### read line by line from csv and save to mongoDB ###
with open(filename, 'r') as f:
    reader = csv.DictReader(f)
    currentCode = None
    
    for i, line in enumerate(reader):
        
        if currentCode == None or currentCode != line['證券代碼']:
            
            if currentCode is not None:
                thisStock = stockCollec.insert_one(copy.deepcopy(p_dict))
                # pp.pprint(thisStock)
            
            p_dict = {'code': None,'history': []}
            currentCode = line['證券代碼']
            p_dict['code'] = line['證券代碼'].split(' ')[0]
            appendHis(p_dict, line)
            
        else:
            appendHis(p_dict, line)

In [56]:
start = datetime.datetime(2017, 11, 8, 0, 0, 0)
end = datetime.datetime(2017, 11, 15, 0, 0, 0)

stockCollec.find_one({'code': '0050'}, {'_id': 0})